In [4]:
%%bash
# Install deps from 
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get update


apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip

# Boost libraries
apt-get install libboost-all-dev

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,105 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [1,079 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.dow

In [5]:
!pip install vizdoom
!pip install ray 
!pip install ray['rllib']
!pip install Ipython --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached vizdoom-1.1.13.tar.gz (15.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for vizdoom: filename=vizdoom-1.1.13-cp37-cp37m-linux_x86_64.whl size=14101151 sha256=b413778309674a485288a792d65eab376b699440ed7b072d3eb6042caf50df3d
  Stored in directory: /root/.cache/pip/wheels/ac/37/ae/8e648023f66bb4c473701f94ce126032ff39ad9759ca0645a7
Successfully built vizdoom
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import vizdoom

In [8]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project'
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append(system_path)

Mounted at /content/drive


In [9]:
print(sys.path)

['/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/GitHub/INM363-Project']


In [10]:
from src.vizdoom_gym.envs.VizDoomEnv import VizdoomEnv
from src.vizdoom_gym.envs.VizDoomEnv_def import VizDoomVeryDenseReward

In [11]:
from ray.tune.registry import register_env
import gym
import os
import ray
import ray.rllib.agents.ppo as ppo
from ray.rllib.algorithms.callbacks import RE3UpdateCallbacks
import shutil

In [12]:
#print("Hello world")

#chkpt_root = "tmp/exa"
chkpt_root = "'/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_dense_scaled_rs_icm"
shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)

#print("here 1")

ray_results = "{}/ray_results/".format(os.getenv("HOME"))
shutil.rmtree(ray_results, ignore_errors=True, onerror=None)

#print("here 2")

ray.shutdown()
print("Shutdown ray")
# start Ray -- add `local_mode=True` here for debugging
ray.init(ignore_reinit_error=True, local_mode=True)
#print("here 3: initialized ray")

# register the custom environment
select_env = "VizDoomVeryDenseReward-v0"

register_env(select_env, lambda config: VizDoomVeryDenseReward())
#register_env(select_env, lambda config: VizdoomEnv())

print("registered environment")


Shutdown ray


/usr/local/lib/python3.7/dist-packages/ray/_private/client_mode_hook.py:105: DeprecationWarning: DeprecationWarning: local mode is an experimental feature that is no longer maintained and will be removed in the future.For debugging consider using Ray debugger. 
  return func(*args, **kwargs)
2022-09-15 13:54:18,967	INFO worker.py:1518 -- Started a local Ray instance.


registered environment


In [ ]:
#%load_ext autoreload
#%autoreload 2
#%reload_ext autoreload

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [15]:
config = ppo.DEFAULT_CONFIG.copy()
config

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': None,
 'env_config': {},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 'num_workers': 2,
 'num_envs_per_worker': 1,
 'sample_collector': ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector,


In [14]:
# configure the environment and create agent
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"
#config["num_workers"] = 1
config["framework"] = "tf"


class RE3Callbacks(RE3UpdateCallbacks, config["callbacks"]):
  pass


#https://github.com/ray-project/ray/blob/c9c3f0745a9291a4de0872bdfa69e4ffdfac3657/rllib/utils/exploration/tests/test_random_encoder.py#L35

config["seed"] = 12345
config["callbacks"] = RE3Callbacks
config["exploration_config"] = {
            "type": "RE3",
            "embeds_dim": 128,
            "beta_schedule": "constant",
            "sub_exploration": {
                "type": "StochasticSampling",
            }
        }

config["seed"] = 12345
config["callbacks"] = RE3Callbacks
config["exploration_config"] = {
    "type": "RE3",
     # the dimensionality of the observation embedding vectors in latent space.
     "embeds_dim": 128,
     "rho": 0.1, # Beta decay factor, used for on-policy algorithm.
     "k_nn": 50, # Number of neighbours to set for K-NN entropy estimation.
     # Configuration for the encoder network, producing embedding vectors from observations.
     # This can be used to configure fcnet- or conv_net setups to properly process any
     # observation space. By default uses the Policy model configuration.
     "encoder_net_config": {
         "fcnet_hiddens": [],
         "fcnet_activation": "relu",
     },
     # Hyperparameter to choose between exploration and exploitation. A higher value of beta adds
     # more importance to the intrinsic reward, as per the following equation
     # `reward = r + beta * intrinsic_reward`
     "beta": 0.2,
     # Schedule to use for beta decay, one of constant" or "linear_decay".
     "beta_schedule": 'constant',
     # Specify, which exploration sub-type to use (usually, the algo's "default"
     # exploration, e.g. EpsilonGreedy for DQN, StochasticSampling for PG/SAC).
     "sub_exploration": {
         "type": "StochasticSampling",
     }
}


agent = ppo.PPOTrainer(config, env=select_env)

print("created agent")

2022-09-15 13:57:35,195	INFO algorithm.py:1872 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-09-15 13:57:35,196	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-09-15 13:57:35,204	INFO algorithm.py:354 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
/usr/local/lib/python3.7/dist-packages/ray/_private/ray_option_utils.py:273: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.0/ray-core/objects/object-spilling.h

:actor_name:RolloutWorker
config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/easy_dense_reward_scaled_rs.wad


2022-09-15 13:57:37,524	ERROR actor.py:984 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=65, ip=172.28.0.2, repr=<ray.rllib.evaluation.rollout_worker._modify_class.<locals>.Class object at 0x7fa9bb05f790>)
  File "/usr/local/lib/python3.7/dist-packages/ray/rllib/evaluation/rollout_worker.py", line 490, in __init__
    self.env = env_creator(copy.deepcopy(self.env_context))
  File "<ipython-input-12-5c6f44b3e151>", line 23, in <lambda>
    register_env(select_env, lambda config: VizDoomVeryDenseReward())
  File "/content/drive/MyDrive/GitHub/INM363-Project/src/vizdoom_gym/envs/VizDoomEnv_def.py", line 11, in __init__
    super(VizDoomVeryDenseReward, self).__init__(1, **kwargs)
  File "/content/drive/MyDrive/GitHub/INM363-Project/src/vizdoom_gym/envs/VizDoomEnv.py", line 79, in __init__
    self.game.init()
vizdoom.vizdoom.SignalException: Signal SIGINT received. ViZDoom instance has been closed.


:actor_name:RolloutWorker
config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/easy_dense_reward_scaled_rs.wad


/usr/local/lib/python3.7/dist-packages/gym/core.py:173: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  "Function `env.seed(seed)` is marked as deprecated and will be removed in the future. "


RayTaskError(AttributeError): ignored

In [ ]:
#config

In [ ]:
status = "{:2d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:4.2f} saved {}"
n_iter = 10

print("started training loop")

# train a policy with RLlib using PPO+ICM
for n in range(n_iter):
  
    result = agent.train()

    #print(i)
    #print(result)
    chkpt_file = agent.save(chkpt_root)

    print(status.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"],
        chkpt_file
    ))

print("ending training loop")

ray.shutdown()
print("shutdown ray")

started training loop
 1 reward  -0.10/ -0.10/ -0.10 len 25.00 saved '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_dense_scaled_rs_icm/checkpoint_000001
 2 reward  -0.10/ -0.10/ -0.10 len 25.00 saved '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_dense_scaled_rs_icm/checkpoint_000002


KeyboardInterrupt: ignored

In [ ]:
 result

{'custom_metrics': {},
 'episode_media': {},
 'num_recreated_workers': 0,
 'info': {'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.10000000149011612,
     'cur_lr': 4.999999873689376e-05,
     'total_loss': 9.908288,
     'policy_loss': -0.09171207,
     'vf_loss': 10.0,
     'vf_explained_var': -1.0,
     'kl': -1.4584478e-10,
     'entropy': 1.4348486e-08,
     'entropy_coeff': 0.0,
     'model': {}},
    'custom_metrics': {},
    'num_agent_steps_trained': 128.0}},
  'num_env_steps_sampled': 8000,
  'num_env_steps_trained': 8000,
  'num_agent_steps_sampled': 8000,
  'num_agent_steps_trained': 8000},
 'sampler_results': {'episode_reward_max': -0.10000000000000006,
  'episode_reward_min': -0.10000000000000006,
  'episode_reward_mean': -0.10000000000000006,
  'episode_len_mean': 25.0,
  'episode_media': {},
  'episodes_this_iter': 160,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_

In [ ]:
print("hello")

In [ ]:
# examine the trained policy
policy = agent.get_policy()
model = policy.model

In [ ]:
model

VisionNetwork(
  (_logits): SlimConv2d(
    (_model): Sequential(
      (0): ZeroPad2d((0, 0, 0, 0))
      (1): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
    )
  )
  (_convs): Sequential(
    (0): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((5, 6, 5, 5))
        (1): Conv2d(3, 16, kernel_size=(12, 16), stride=(7, 9))
        (2): ReLU()
      )
    )
    (1): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((1, 1, 1, 2))
        (1): Conv2d(16, 32, kernel_size=(6, 6), stride=(4, 4))
        (2): ReLU()
      )
    )
    (2): SlimConv2d(
      (_model): Sequential(
        (0): Conv2d(32, 256, kernel_size=(9, 9), stride=(1, 1))
        (1): ReLU()
      )
    )
  )
  (_value_branch_separate): Sequential(
    (0): SlimConv2d(
      (_model): Sequential(
        (0): ZeroPad2d((5, 6, 5, 5))
        (1): Conv2d(3, 16, kernel_size=(12, 16), stride=(7, 9))
        (2): ReLU()
      )
    )
    (1): SlimConv2d(
      (_model): Sequential(
        (0): 

In [ ]:
system_path = '/content/drive/MyDrive/GitHub/INM363-Project/src' #need this to load vizdoom module 
sys.path.append(system_path)

In [ ]:
agent.restore(chkpt_file)
env = gym.make('VizDoomVeryDenseReward-v0')

2022-09-02 13:04:28,524	INFO trainable.py:669 -- Restored on 172.28.0.2 from checkpoint: '/content/drive/MyDrive/GitHub/INM363-Project/tmp/easy_dense_scaled_rs_icm/checkpoint_000005
2022-09-02 13:04:28,527	INFO trainable.py:677 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': None, '_time_total': 5707.500942707062, '_episodes_total': 860}


config file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/very_dense_reward.cfg
scenario file: /content/drive/MyDrive/GitHub/INM363-Project/scenarios/custom/easy_dense_reward_scaled_rs.wad


In [ ]:
state = env.reset()
sum_reward = 0
n_step = 30

for step in range(n_step):
    action = agent.compute_action(state)
    print(action)
    state, reward, done, info = env.step(action)
    sum_reward += reward

    #env.render()

    if done == 1:
        # report at the end of each episode
        print("cumulative reward", sum_reward)
        print(f"total steps: {step}")
        state = env.reset()
        sum_reward = 0
        break

if done == 0:
  print("cumulative reward", sum_reward)
  print(f"total steps: {step}")


1
1
1
1
2
1
2
2
cumulative reward 4.970000000000001
total steps: 7


In [ ]:
#chkpt_root = "'/content/drive/MyDrive/GitHub/INM363-Project/tmp/exa"
#shutil.rmtree(chkpt_root, ignore_errors=True, onerror=None)

In [ ]:
"""
print(result)

chkpt_file = agent.save(chkpt_root)

print(status.format(
  n + 1,
  result["episode_reward_min"],
  result["episode_reward_mean"],
  result["episode_reward_max"],
  result["episode_len_mean"],
  chkpt_file
))
"""